In [1]:
import numpy as np
import pandas as pd
import keras
from keras import layers, models
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, Adamax, SGD
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files
import math
import random

# Ladataan data CSV-tiedostosta
file_path = "data_from_mysql_where_g160.csv"
data = pd.read_csv(file_path)

# Tarkastetaan datan rakenne
#print(data.head())

# Suodatetaan pois kaikki rivit, joissa 'sensorvalue_d' on 0
data_filtered = data[data['sensorvalue_d'] != 0]

x_data = data_filtered[['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c']].values  # x, y, z
y_data = data_filtered['sensorvalue_d'].values # suunta

# Lasketaan luokkien esiintymät
class_counts = data_filtered['sensorvalue_d'].value_counts()
print("Luokkien esiintymät:\n", class_counts)

# Skaalaus [0, 1] väliin ilman NumPy:n vektorilaskentaa
def scale_to_unit_interval(data):
    # Alustetaan listat minimi- ja maksimiarvoja varten
    min_vals = [float('inf')] * len(data[0])  # Asetetaan alkuarvoksi "ääretön" jokaiselle sarakkeelle
    max_vals = [float('-inf')] * len(data[0])  # Asetetaan alkuarvoksi "-ääretön" jokaiselle sarakkeelle

    # Lasketaan minimi- ja maksimiarvot sarakkeittain
    for row in data:
        for col_idx, value in enumerate(row):
            if value < min_vals[col_idx]:
                min_vals[col_idx] = value
            if value > max_vals[col_idx]:
                max_vals[col_idx] = value

    # Lasketaan skaalatut arvot
    scaled_data = []
    for row in data:
        scaled_row = []
        for col_idx, value in enumerate(row):
            range_val = max_vals[col_idx] - min_vals[col_idx]
            if range_val == 0:
                range_val = 1  # Vältetään nollalla jakamista
            scaled_value = (value - min_vals[col_idx]) / range_val
            scaled_row.append(scaled_value)
        scaled_data.append(scaled_row)

    return scaled_data

# Skaalataan syötteet
x_data_scaled = scale_to_unit_interval(x_data)

# Suuntaa on 6 luokkaa
num_classes = 6
y_data = keras.utils.to_categorical(y_data - 1, num_classes)

x_train, x_test, y_train, y_test = train_test_split(x_data_scaled, y_data, test_size=0.2, random_state=42)

print(f"x_train shape: ({len(x_train)}, {len(x_train[0])})")
print(f"x_test shape: ({len(x_test)}, {len(x_test[0])})")

# Muutetaan x_train ja x_test takaisin DataFrameiksi alkuperäisillä sarakenimillä
x_train = pd.DataFrame(x_train, columns=['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c'])
x_test = pd.DataFrame(x_test, columns=['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c'])

# Funktio kohinan lisäämiseen
def add_noise_to_data(df, columns, noise_factor=0.05):
    noisy_df = df.copy()  # Säilytä alkuperäinen DataFrame
    for column in columns:
        std = df[column].std()
        noise = [random.gauss(0, noise_factor * std) for _ in range(len(df))]  # Korvattu NumPy:lla
        noisy_df[column] += noise  # Lisää kohina sarakkeeseen
    return noisy_df

# Lisää kohinaa harjoitus- ja testidataan
x_train_noisy = add_noise_to_data(x_train, ['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c'])
x_test_noisy = add_noise_to_data(x_test, ['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c'])

# Määritellään malli
model = keras.Sequential(
    [
        keras.Input(shape=(x_train_noisy.shape[1],)),  # Syötemuoto (x, y, z)
        layers.Dense(64, activation="relu"),  # Ensimmäinen tiheä kerros
        layers.Dropout(0.2),  # Dropout, jotta ylikoulutus ei tapahdu
        layers.Dense(32, activation="relu"),  # Toinen tiheä kerros
        layers.LeakyReLU(negative_slope=0.2),  # Vaihtoehto ReLU:lle
        layers.Dense(num_classes, activation="softmax"),  # Lopullinen luokittelukerros
    ]
)

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Koulutetaan mallia
batch_size = 128
epochs = 20

optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train_noisy, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping])

# Arvioidaan malli testidatalla
score = model.evaluate(x_test_noisy, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Tallennetaan malli (painot ja rakenne)
model.save('my_model.keras')

y_pred = model.predict(x_test_noisy)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_classes, y_pred_classes))
print("")

# Osa 2

# Haetaan mallin painot
weights = model.get_weights()

# Painot ovat lista, jossa on numpy-taulukoita
for idx, weight in enumerate(weights):
    print(f"Painojen ja biasin {idx} muoto: {weight.shape}")
    print("")
    #print(f"Painot ja bias {idx}:", weight)

# Aktivointifunktio ReLU
def relu(x):
    if isinstance(x, list):  # Jos x on lista
        return [max(0, val) for val in x]
    return max(0, x)  # Jos x on yksittäinen arvo

# Aktivointifunktio Softmax
def softmax(z):
    exp_values = [2.718 ** i for i in z]  # Eksponenttiarvot
    total = sum(exp_values)  # Lasketaan summan eksponentit
    return [exp_value / total for exp_value in exp_values]  # Jaa eksponenttiarvot kokonaisuudella

# Syöte
for i in range(len(data_filtered)):
    # Haetaan kunkin rivin arvot
    x = data_filtered['sensorvalue_a'].values[i]
    y = data_filtered['sensorvalue_b'].values[i]
    z = data_filtered['sensorvalue_c'].values[i]

    # Muutetaan syöte oikeaan muotoon
    input_data = [x, y, z]

#print("Input data shape:", input_data.shape)
print("Input data shape:", len(input_data))

# Skaalataan syöte
input_data_scaled = scale_to_unit_interval([input_data])  # Muutetaan lista sisään

print("Input data scaled shape:", len(input_data_scaled))

# Oikeat painot ja biasit
weights_0, bias_0 = weights[0], weights[1]
weights_1, bias_1 = weights[2], weights[3]
weights_2, bias_2 = weights[4], weights[5]

# Etenee syötteestä piilokerrosten kautta ulostuloon
def forward_propagation(input_data_scaled):
    # Piilokerros 1
    z0 = []
    for k in range(len(weights_0[0])):  # Käydään läpi piilokerroksen neuronit
        z0_value = sum(input_data_scaled[0][i] * weights_0[i][k] for i in range(len(input_data_scaled[0]))) + bias_0[k]
        z0.append(z0_value)
    a0 = [relu(z) for z in z0]  # Aktivointi

    # Piilokerros 2
    z1 = []
    for k in range(len(weights_1[0])):
        z1_value = sum(a0[i] * weights_1[i][k] for i in range(len(a0))) + bias_1[k]
        z1.append(z1_value)
    a1 = [relu(z) for z in z1]  # Aktivointi

    # Ulostulokerros
    z2 = []
    for k in range(len(weights_2[0])):  # Kolmannen kerroksen laskentaa (ulostulo)
        z2_value = sum(a1[i] * weights_2[i][k] for i in range(len(a1))) + bias_2[k]
        z2.append(z2_value)
    output = softmax(z2)  # Softmax aktivointi

    return output

# Asetetaan NumPy:n tulostustapa niin, että ei käytetä tieteellistä merkintää
np.set_printoptions(precision=6, suppress=True)

# Lasketaan tulos syötteelle (x, y, z)
result = forward_propagation(input_data_scaled)
print("\nVerkon ulostulo: (forward_propagation):", result)

# Oikea syötemuoto model.predict
input_data_scaled = scale_to_unit_interval([input_data])  # Muutetaan lista sisään

# Varmistetaan, että syöte on oikeassa muodossa
input_data_scaled = np.array(input_data_scaled)  # Muutetaan NumPy-taulukoksi
input_data_scaled = input_data_scaled.reshape(1, -1)  # Muotoillaan se (1, 3) muotoon

# Lasketaan ennuste koulutetulla mallilla
prediction = model.predict(input_data_scaled)
print("\nEnnuste (model.predict):", prediction)

# Lasketaan ero tulosten välillä
result = result  # Varmistetaan, että result on lista
prediction = prediction[0].tolist()  # Muutetaan prediction listaksi

# Lasketaan ero
difference = [abs(r - p) for r, p in zip(result, prediction)]  # Lasketaan itseisarvoero

# Tulostetaan ero desimaaleina ilman tieteellistä muotoa
print("\nEro (absoluuttinen ero result ja prediction välillä):")
for i, diff in enumerate(difference):
    print(f"Ero {i+1}: {diff:.8f}")  # Tulostetaan desimaalimuodossa, pyöristettynä 8 desimaaliin

# Keskimääräinen ero
mean_difference = sum(difference) / len(difference)
# Tulostetaan keskimääräinen ero desimaaleina ilman tieteellistä muotoa
print("\nKeskimääräinen ero: ", f"{mean_difference:.8f}")

# Tallennetaan painot ja biasit header-tiedostoon
header_file = "neuroverkonKertoimet2.h"

with open(header_file, "w") as f:
    f.write("#ifndef NEUROVERKONKERTOIMET_H\n")
    f.write("#define NEUROVERKONKERTOIMET_H\n\n")

    # Kirjoitetaan painot ja biasit jokaiselle kerrokselle
    for idx, weight in enumerate(weights):
        if len(weight.shape) == 2:  # Painot (matriisi)
            f.write(f"float weights_{idx}[{weight.shape[0]}][{weight.shape[1]}] = {{\n")
            for row in weight:
                f.write("    {" + ", ".join(map(str, row)) + "},\n")
            f.write("};\n\n")
        elif len(weight.shape) == 1:  # Bias (vektori)
            f.write(f"float biases_{idx}[{weight.shape[0]}] = {{")
            f.write(", ".join(map(str, weight)))
            f.write("};\n\n")

    f.write("#endif // NEUROVERKONKERTOIMET_H\n")


    # Lataa malli
model = tf.keras.models.load_model('my_model.keras')

# Muunna TensorFlow Lite -malliksi
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Tallenna muunnos
with open('oma_malli.tflite', 'wb') as f:
    f.write(tflite_model)



Luokkien esiintymät:
 sensorvalue_d
5.0    266
6.0    242
3.0    218
2.0    217
1.0    193
4.0    189
Name: count, dtype: int64
x_train shape: (1060, 3)
x_test shape: (265, 3)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,534 (9.90 KB)

 Trainable params: 2,534 (9.90 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 408ms/step - accuracy: 0.1290 - loss: 1.7768 - val_accuracy: 0.0000e+00 - val_loss: 1.7447
Epoch 2/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.3021 - loss: 1.7330 - val_accuracy: 0.6698 - val_loss: 1.6967
Epoch 3/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4876 - loss: 1.6887 - val_accuracy: 0.7925 - val_loss: 1.6439
Epoch 4/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7016 - loss: 1.6348 - val_accuracy: 0.8019 - val_loss: 1.5845
Epoch 5/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7701 - loss: 1.5764 - val_accuracy: 0.8019 - val_loss: 1.5165
Epoch 6/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8053 - loss: 1.5026 - val_accuracy: 0.8019 - val_loss: 1.4382
Epoch 7/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8188 - loss: 1.4279 - val_accuracy: 0.8019 - val_loss: 1.3470
Epoch 8/20
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8438 - loss: 1.3342 - val_accuracy: 0.8019 - val_loss: 1.2490
